In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 51.7 MB/s eta 0:00:00


In [ ]:
import os

import copy
import numpy as np
import pandas as pd
#import polars as pl
import matplotlib
import matplotlib.pyplot as plt
import random
from itertools import cycle
from functools import partial
from typing import Union, List, Optional
#from sklearn import preprocessing
#from sklearn.pipeline import Pipeline
#from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import KFold
#from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
#from sklearn import model_selection
#from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
#import statsmodels.formula.api as smf
#import optuna
from torch import nn
#from scipy.stats import norm
#from scipy import stats

import warnings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import torch
import torch.nn.functional as F
#import pytorch_lightning as pl
#from pytorch_lightning.loggers import WandbLogger
import scipy
#from skorch import NeuralNetRegressor

In [ ]:
#train set mean and std
train_mean = np.array([2.5967546 , 1.0034839 , 0.04016334, 0.02489665, 0.25563124,
       1.1303542 , 1.140971  , 0.434731  , 0.00295969], dtype=np.float32)
train_std = np.array([1.3872927 , 0.5756184 , 0.02310472, 0.01444912, 0.14133842,
       0.5279417 , 0.5266676 , 0.27412835, 0.517801  ], dtype=np.float32)
class PricingNetwork3(nn.Module):
  def __init__(self, mean=train_mean, std=train_std, neurons=64, x_features=9):
  #def __init__(self, neurons=64, x_features=5):
    super(PricingNetwork3,self).__init__()
    self.mean = mean
    self.std = std
    self.bottleneck = nn.Sequential(nn.Linear(x_features, neurons), nn.LeakyReLU(),
                                     nn.Linear(neurons, neurons), nn.LeakyReLU(),
                                     nn.Linear(neurons, neurons), nn.LeakyReLU(),
                                     nn.Linear(neurons, neurons), nn.LeakyReLU(),
                                     #nn.Linear(neurons, neurons), nn.LeakyReLU(),
                                     nn.Linear(neurons, neurons), nn.LeakyReLU())
    #self.bottleneck = nn.ModuleList([nn.Linear(x_features, neurons), nn.LeakyReLU()] + [nn.Linear(neurons, neurons), nn.LeakyReLU()] * 4)
    self.final_layer = nn.Linear(neurons, 1)

  def forward(self, X, **kwargs):
    X = X - self.mean / self.std
    X = self.bottleneck(X)
    X  = self.final_layer(X)
    return X


mm2 = PricingNetwork3()
mm2.load_state_dict(torch.load('/content/drive/MyDrive/practicum_data-American_Option/amer_transform100k.pt'))

<All keys matched successfully>

In [ ]:
def parse_input(X_, params_):
  new_input = torch.empty((X_.shape[0], X_.shape[1]+params_.shape[0]))
  new_input[:,:4] = X_
  new_input[:,4:] = torch.Tensor(params_)
  return new_input


In [ ]:
%%time
print("time usage for ANN model")
for _ in range(5):
  param = np.array([ 9.225e-05 , 8.494e-05 , 7.719e-01 ,5.071e-01, -4.983e-01])
  objective_function(param)

time usage for ANN model
CPU times: user 40.4 ms, sys: 0 ns, total: 40.4 ms
Wall time: 85.3 ms


In [ ]:
with torch.no_grad():https://accounts.google.com/SignOutOptions?hl=en&continue=https://colab.research.google.com/drive/1DEupwRbwEhfzXDXkKTdpTSVsaGx18CXZ&ec=GBRAqQM
  calibrated = np.array([ 2.771e-02 ,9.073e-04 ,1.136e+00 ,6.358e-01,-4.999e-01])
  real = np.array([0.01, 0.02,	1.0,	0.1,	0.2	]) #real Heston Parameters
  print("Objective Result using calibrated params:",objective_function(calibrated))
  print("Objective Result using real answer:",objective_function(real))

Objective Result using calibrated params: 0.003206874
Objective Result using real answer: 0.005562815


In [ ]:
def objective_function(X_input,params):
    # params is a numpy array containing the Heston model parameters

    # Use the neural network to predict the option price
    with torch.no_grad():
      price_pred = mm2(parse_input(X_input, params)).flatten().numpy()

    # Calculate the squared difference between the predicted and observed prices
    diff = price_pred - price_ft
    squared_diff = np.mean(np.abs(diff))

    return squared_diff


In [ ]:
def Feller(params):
  '''
  params: a list of Heston model parameters [v0, theta, kappa, sigma, rho]
  The result should be negative.
  '''
  _, theta, kappa, sigma, _ = params
  return sigma**2 - 2*kappa*theta

cons = {'type':'ineq', 'fun':Feller}

In [ ]:
params1_0 = np.array([0.02, 0.02, 1.3, 0.1, -0.3])
params2_0 = np.array([0.02, 0.01, 0.8, 0.2, 0.1])
params3_0 = np.array([0.04, 0.05, 1.3, 0.2, -0.7])
cal_res = []
for (i, param) in zip([0, 1, 2], [params1_0, params2_0, params3_0]):
  from scipy.optimize import NonlinearConstraint
  e_ft1 = pd.read_csv(f'/content/drive/MyDrive/practicum_data/calibrate_5k_param{i}.csv', index_col=0)
  X_input = torch.tensor(e_ft1.values[:,:4].astype(np.float32))
  price_ft = e_ft1['eurocall_ft'].values.astype(np.float32)
  nlc = NonlinearConstraint(Feller, -np.inf, 0)
  cal_res.append(scipy.optimize.differential_evolution(objective_function, bounds=[(1e-15,0.5), (1e-15,2), (1e-15,2), (1e-15,2), (-1,2)], x0=param,constraints=nlc).x)
  print(f"{i} done!")

/usr/local/lib/python3.9/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


0 done!


/usr/local/lib/python3.9/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


1 done!


/usr/local/lib/python3.9/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


2 done!


In [ ]:
def price_diff(X_input,params):
  with torch.no_grad():
    price_pred = mm2(parse_input(X_input, params)).flatten().numpy()

    # Calculate the squared difference between the predicted and observed prices
  diff = price_pred - price_ft
  squared_diff = np.mean(np.abs(diff))
  return squared_diff

In [ ]:
cal_res

[array([ 0.06945654,  0.10392993,  0.59085353,  0.34717689, -0.66581399]),
 array([ 0.01530431,  0.31101196,  0.21943453,  0.36608919, -0.57734179]),
 array([ 0.07556475,  0.31534085,  0.34879871,  0.46755522, -0.77659388])]

In [ ]:
mean_ = 0.
for (i, param) in zip([0, 1, 2], cal_res):
  e_ft1 = pd.read_csv(f'/content/drive/MyDrive/practicum_data/calibrate_5k_param{i}.csv', index_col=0)
  X_input = torch.tensor(e_ft1.values[:,:4].astype(np.float32))
  price_ft = e_ft1['eurocall_ft'].values.astype(np.float32)
  mean_ += price_diff(X_input, param)

mean_ / 3.

0.00391547071437041

In [ ]:
def report_calibration(initial, actual, calibrated):
  initial1 = initial + [Feller(initial)]

  actual1 = actual + [Feller(actual)]
  calibrated1 = np.append(calibrated, Feller(calibrated))
  report = pd.DataFrame({"Initial": initial1, "True": actual1, "Calibrated": calibrated1},
                        index=['v0', 'theta', 'kappa', 'sigma', 'rho', 'Feller']).round(5).T
  return report

In [ ]:
report_calibration(params0.tolist(),[0.01, 0.02,	1.0,	0.1,	0.2	], np.array([ 0. ,  0. , 0.772,  0.507, -0.498]))

,v0,theta,kappa,sigma,rho,Feller
Initial,0.02,0.01,0.800,0.200,0.100,0.02400
True,0.01,0.02,1.000,0.100,0.200,-0.03000
Calibrated,0.00,0.00,0.772,0.507,-0.498,0.25705


In [ ]:
actual1 = [0.04,0.04,1.5,0.3,-0.5]
actual2 = [0.01,0.02,1.,0.1,0.2]
actual3 = [0.06,0.06,1.5,0.3,-0.6]

In [ ]:
report_calibration(params1_0.tolist(),actual1, cal_res[0])

,v0,theta,kappa,sigma,rho,Feller
Initial,0.02000,0.0200,1.30000,0.1000,-0.30000,-0.04200
True,0.04000,0.0400,1.50000,0.3000,-0.50000,-0.03000
Calibrated,0.00809,0.0602,0.82162,0.3133,-0.99599,-0.00077


In [ ]:
report_calibration(params2_0.tolist(),actual2, cal_res[1])

,v0,theta,kappa,sigma,rho,Feller
Initial,0.02000,0.01000,0.80000,0.20000,0.10000,0.02400
True,0.01000,0.02000,1.00000,0.10000,0.20000,-0.03000
Calibrated,0.00001,0.04723,0.58655,0.23535,-0.99965,-0.00001


In [ ]:
cal_res

[array([ 0.00809362,  0.06020034,  0.82161514,  0.313297  , -0.99599333]),
 array([ 5.05111577e-06,  4.72288805e-02,  5.86545156e-01,  2.35353878e-01,
        -9.99654886e-01]),
 array([ 0.05631509,  0.0688414 ,  1.03246575,  0.37663803, -0.88684139])]

In [ ]:
report_calibration(params3_0.tolist(), actual3, cal_res[2])

,v0,theta,kappa,sigma,rho,Feller
Initial,0.04000,0.05000,1.30000,0.20000,-0.70000,-0.0900
True,0.06000,0.06000,1.50000,0.30000,-0.60000,-0.0900
Calibrated,0.05632,0.06884,1.03247,0.37664,-0.88684,-0.0003


In [ ]:
def objective_function2(params):
    # params is a numpy array containing the Heston model parameters

    # Use the neural network to predict the option price
    with torch.no_grad():
      price_pred = mm2(parse_input(X_input, params)).flatten().numpy()

    # Calculate the squared difference between the predicted and observed prices
    diff = price_pred - price_ft
    squared_diff = np.mean(diff))

    return squared_diff


In [ ]:
for (i, param) in zip([0, 1, 2], cal_res):
  e_ft1 = pd.read_csv(f'/content/drive/MyDrive/practicum_data/calibrate_5k_param{i}.csv', index_col=0)
  #X_input = torch.tensor(e_ft1.values[:,:4].astype(np.float32))
  price_ft = e_ft1['eurocall_ft'].values.astype(np.float32)
  print(objective_function2(param))

0.0054077134
0.006625282
0.002279052
